In [138]:
import os
import helpers as h
import features as f
from glob import glob
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt

# Training XGBoost with best parameters on minimum features

# Helper functions assume the directory structure to be:
# - project_data/
#     - train/
#         - sample1.wav/
#         - sample2.wav/
#     - test_clean/
#         - sample1.wav/
#         - sample2.wav/
#     - test_noisy/
#         - sample1.wav/
#         - sample2.wav/
# - features/


In [ ]:
# In spite of the name, this is not a grid search. It just runs the feature
# extraction function with the given parameters.
param_grid = {
    "logpower": [2],
    "n_mels": [128],
    "mean": [True],
    "std": [True],
    "variance": [True],
    "delta": [True],
    "delta2": [True],
    "delta3": [True],
    "hop_length": [256],
    "n_fft": [2048],
    }

h.grid_search(param_grid, f.extract_log_mel_filterbank)


Preparing datasets with feature extractor: extract_log_mel_filterbank
Parameter values: {'logpower': 2, 'n_mels': 128, 'mean': True, 'std': True, 'variance': True, 'delta': True, 'delta2': True, 'delta3': True, 'hop_length': 256, 'n_fft': 2048}


Processing Noisy Test Files: 100%|██████████| 3238/3238 [00:48<00:00, 66.74it/s]


Features and labels saved to features/extract_log_mel_filterbank_logpower-2_n_mels-128_mean-True_std-True_variance-True_delta-True_delta2-True_delta3-True_hop_length-256_n_fft-2048.npz


In [4]:
param_grid = {
    "n_mfcc": [13],
    "n_fft": [2048],
    "hop_length": [256],
}

h.grid_search(param_grid, f.generate_mfcc)


Preparing datasets with feature extractor: generate_mfcc
Parameter values: {'n_mfcc': 13, 'n_fft': 2048, 'hop_length': 256}


Processing Noisy Test Files: 100%|██████████| 3238/3238 [00:28<00:00, 115.59it/s]


Features and labels saved to features/generate_mfcc_n_mfcc-13_n_fft-2048_hop_length-256.npz


In [ ]:
param_grid = {
    "hop_length": [512],
    "frame_length": [2048],
}

h.grid_search(param_grid, f.compute_rms)


Preparing datasets with feature extractor: compute_rms
Parameter values: {'hop_length': 512, 'frame_length': 2048}


Processing Noisy Test Files: 100%|██████████| 3238/3238 [00:16<00:00, 200.36it/s]


Features and labels saved to features/compute_rms_hop_length-512_frame_length-2048.npz


In [ ]:
# Combining features

feature_folder = 'features'
feature_files = glob(os.path.join(feature_folder, "*.npz"))

combined_data_per_name = {}

for file in feature_files:
    data = np.load(os.path.join(file))
    for key in data:
        if key not in combined_data_per_name:
            combined_data_per_name[key] = []
        combined_data_per_name[key].append(data[key])

if not os.path.isdir(f"{feature_folder}/final"):
    os.makedirs(f"{feature_folder}/final")

final_feats_filename = os.path.join(f"{feature_folder}/final", 'final_combined_features.npz')
h.combine_and_save_features(combined_data_per_name, final_feats_filename)


Features saved to features/final/final_combined_features.npz


In [ ]:
final_features = np.load(final_feats_filename)

# model = xgb.XGBClassifier(device="cuda")
model = xgb.XGBClassifier()
model.fit(final_features["train_features"],final_features["train_labels"],)

print("Train Clean Acc =", np.sum(final_features["train_labels"]==model.predict(final_features["train_features"]))/len(final_features["train_labels"]))
print("Test Clean Acc =", np.sum(final_features["test_clean_labels"]==model.predict(final_features["test_clean_features"]))/len(final_features["test_clean_labels"]))
print("Test Noisy Acc =", np.sum(final_features["test_noisy_labels"]==model.predict(final_features["test_noisy_features"]))/len(final_features["test_noisy_labels"]))


Train Clean Acc = 1.0
Test Clean Acc = 0.7125937623371497
Test Noisy Acc = 0.7362569487337863
